<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Functions" data-toc-modified-id="Functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Import-and-Analysis" data-toc-modified-id="Import-and-Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import and Analysis</a></span><ul class="toc-item"><li><span><a href="#Dataset-description:" data-toc-modified-id="Dataset-description:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Dataset description:</a></span></li><li><span><a href="#Data-Investigation-and-Cleaning" data-toc-modified-id="Data-Investigation-and-Cleaning-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Data Investigation and Cleaning</a></span><ul class="toc-item"><li><span><a href="#Administrative" data-toc-modified-id="Administrative-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Administrative</a></span></li><li><span><a href="#Administrative-Duration" data-toc-modified-id="Administrative-Duration-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Administrative Duration</a></span></li><li><span><a href="#Informational" data-toc-modified-id="Informational-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Informational</a></span></li><li><span><a href="#Informational-Duration" data-toc-modified-id="Informational-Duration-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Informational Duration</a></span></li><li><span><a href="#Product-Related" data-toc-modified-id="Product-Related-2.2.5"><span class="toc-item-num">2.2.5&nbsp;&nbsp;</span>Product Related</a></span></li><li><span><a href="#Product-Related-Duration" data-toc-modified-id="Product-Related-Duration-2.2.6"><span class="toc-item-num">2.2.6&nbsp;&nbsp;</span>Product Related Duration</a></span></li><li><span><a href="#Bounce-Rates" data-toc-modified-id="Bounce-Rates-2.2.7"><span class="toc-item-num">2.2.7&nbsp;&nbsp;</span>Bounce Rates</a></span></li><li><span><a href="#Exit-Rates" data-toc-modified-id="Exit-Rates-2.2.8"><span class="toc-item-num">2.2.8&nbsp;&nbsp;</span>Exit Rates</a></span></li><li><span><a href="#Page-Values" data-toc-modified-id="Page-Values-2.2.9"><span class="toc-item-num">2.2.9&nbsp;&nbsp;</span>Page Values</a></span></li><li><span><a href="#Special-Day" data-toc-modified-id="Special-Day-2.2.10"><span class="toc-item-num">2.2.10&nbsp;&nbsp;</span>Special Day</a></span></li><li><span><a href="#Month" data-toc-modified-id="Month-2.2.11"><span class="toc-item-num">2.2.11&nbsp;&nbsp;</span>Month</a></span></li><li><span><a href="#Operating-Systems" data-toc-modified-id="Operating-Systems-2.2.12"><span class="toc-item-num">2.2.12&nbsp;&nbsp;</span>Operating Systems</a></span></li><li><span><a href="#Browser" data-toc-modified-id="Browser-2.2.13"><span class="toc-item-num">2.2.13&nbsp;&nbsp;</span>Browser</a></span></li><li><span><a href="#Region" data-toc-modified-id="Region-2.2.14"><span class="toc-item-num">2.2.14&nbsp;&nbsp;</span>Region</a></span></li><li><span><a href="#Traffic-type" data-toc-modified-id="Traffic-type-2.2.15"><span class="toc-item-num">2.2.15&nbsp;&nbsp;</span>Traffic type</a></span></li><li><span><a href="#Visitor-Type" data-toc-modified-id="Visitor-Type-2.2.16"><span class="toc-item-num">2.2.16&nbsp;&nbsp;</span>Visitor Type</a></span></li><li><span><a href="#Weekend" data-toc-modified-id="Weekend-2.2.17"><span class="toc-item-num">2.2.17&nbsp;&nbsp;</span>Weekend</a></span></li><li><span><a href="#Revenue" data-toc-modified-id="Revenue-2.2.18"><span class="toc-item-num">2.2.18&nbsp;&nbsp;</span>Revenue</a></span></li><li><span><a href="#Data-Types" data-toc-modified-id="Data-Types-2.2.19"><span class="toc-item-num">2.2.19&nbsp;&nbsp;</span>Data Types</a></span></li></ul></li><li><span><a href="#Data-visualization" data-toc-modified-id="Data-visualization-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Data visualization</a></span></li><li><span><a href="#Data-Transformation" data-toc-modified-id="Data-Transformation-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Data Transformation</a></span><ul class="toc-item"><li><span><a href="#Transforming-columns" data-toc-modified-id="Transforming-columns-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Transforming columns</a></span><ul class="toc-item"><li><span><a href="#Operating-Systems" data-toc-modified-id="Operating-Systems-2.4.1.1"><span class="toc-item-num">2.4.1.1&nbsp;&nbsp;</span>Operating Systems</a></span></li><li><span><a href="#Browser" data-toc-modified-id="Browser-2.4.1.2"><span class="toc-item-num">2.4.1.2&nbsp;&nbsp;</span>Browser</a></span></li><li><span><a href="#Traffic-Type" data-toc-modified-id="Traffic-Type-2.4.1.3"><span class="toc-item-num">2.4.1.3&nbsp;&nbsp;</span>Traffic Type</a></span></li></ul></li><li><span><a href="#Data-encoding" data-toc-modified-id="Data-encoding-2.4.2"><span class="toc-item-num">2.4.2&nbsp;&nbsp;</span>Data encoding</a></span></li></ul></li><li><span><a href="#Missing-Values" data-toc-modified-id="Missing-Values-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Missing Values</a></span></li><li><span><a href="#Correlation" data-toc-modified-id="Correlation-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Correlation</a></span><ul class="toc-item"><li><span><a href="#Correlation-insights:" data-toc-modified-id="Correlation-insights:-2.6.1"><span class="toc-item-num">2.6.1&nbsp;&nbsp;</span>Correlation insights:</a></span></li></ul></li></ul></li><li><span><a href="#Individual-work-to-organize" data-toc-modified-id="Individual-work-to-organize-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Individual work to organize</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Administrative_duration" data-toc-modified-id="Administrative_duration-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Administrative_duration</a></span></li><li><span><a href="#BounceRates" data-toc-modified-id="BounceRates-3.0.2"><span class="toc-item-num">3.0.2&nbsp;&nbsp;</span>BounceRates</a></span></li><li><span><a href="#OperatingSystems" data-toc-modified-id="OperatingSystems-3.0.3"><span class="toc-item-num">3.0.3&nbsp;&nbsp;</span>OperatingSystems</a></span></li><li><span><a href="#Weekend" data-toc-modified-id="Weekend-3.0.4"><span class="toc-item-num">3.0.4&nbsp;&nbsp;</span>Weekend</a></span></li><li><span><a href="#Kristina:" data-toc-modified-id="Kristina:-3.0.5"><span class="toc-item-num">3.0.5&nbsp;&nbsp;</span>Kristina:</a></span><ul class="toc-item"><li><span><a href="#Informational" data-toc-modified-id="Informational-3.0.5.1"><span class="toc-item-num">3.0.5.1&nbsp;&nbsp;</span>Informational</a></span></li><li><span><a href="#Exit-Rates" data-toc-modified-id="Exit-Rates-3.0.5.2"><span class="toc-item-num">3.0.5.2&nbsp;&nbsp;</span>Exit Rates</a></span></li><li><span><a href="#Browser" data-toc-modified-id="Browser-3.0.5.3"><span class="toc-item-num">3.0.5.3&nbsp;&nbsp;</span>Browser</a></span></li><li><span><a href="#Revenue" data-toc-modified-id="Revenue-3.0.5.4"><span class="toc-item-num">3.0.5.4&nbsp;&nbsp;</span>Revenue</a></span></li></ul></li><li><span><a href="#Isaac:" data-toc-modified-id="Isaac:-3.0.6"><span class="toc-item-num">3.0.6&nbsp;&nbsp;</span>Isaac:</a></span></li><li><span><a href="#Pau:" data-toc-modified-id="Pau:-3.0.7"><span class="toc-item-num">3.0.7&nbsp;&nbsp;</span>Pau:</a></span></li><li><span><a href="#Sosa:" data-toc-modified-id="Sosa:-3.0.8"><span class="toc-item-num">3.0.8&nbsp;&nbsp;</span>Sosa:</a></span></li><li><span><a href="#MODEL-APPLICATION-:-K-Nearest-Neighbors¶" data-toc-modified-id="MODEL-APPLICATION-:-K-Nearest-Neighbors¶-3.0.9"><span class="toc-item-num">3.0.9&nbsp;&nbsp;</span>MODEL APPLICATION : K-Nearest Neighbors¶</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


# Functions

In [ ]:
## Function to explain NA values in a column:

def NA_values(series):
    '''
    Function that takes a Pandas Series and returns a print statement explaining NAs and % of the column
    '''
    
    return print(f'Column name: {series.name}\nTotal values: {int(series.count())}\nNA values: {series.isna().sum()}\n% of NA values: {round(series.isna().mean() * 100,2)}%')

In [ ]:
## Function to get the information about the outliers of a column:

def iqr(dataset, series):
    """
    Function takes dataset and column and returns the information about the outliers.
    Input:
        - dataset, e.g. data
        - series, e.g. data.Price, important to use this format!
    """
    Q1 = np.percentile(series, 25)
    Q3 = np.percentile(series, 75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    total_outliers = len(dataset.loc[(series > upper) | (series < lower)])
    percent_outliers = total_outliers / len(dataset) * 100
    
    return print(f'Column: {series.name}\nLower outliers: all values lower than {round(lower, 3)}\nUpper outliers: all values higher than {round(upper, 3)}\nTotal number of rows with outliers: {total_outliers}\n% of outliers: {round(percent_outliers, 2)}%')

# Import and Analysis

In [ ]:
original = pd.read_csv('data/online_shoppers_intention_DATAPTDIC19.csv', sep=',', index_col=0)

In [ ]:
data = original.copy()

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

## Dataset description:
 - Administrative:`float`. Administrative Value. `yet to identify`.
 - Administrative_Duration: `object`. Duration in Administrative Page. `Identify values and change dtype accordingly`. 
 - Informational: `float`. Informational Value. `yet to identify` 
 - Informational_Duration: `object`. Duration in Informational Page. `Identify values and change dtype accordingly.`
 - ProductRelated: `float.` Product Related Value. `yet to identify` 
 - ProductRelated_Duration: `float`. Duration in Product Related Page. 
 - BounceRates: `float`. Bounce Rates of a web page. Percentages. The value of "Bounce Rate" feature for a web page refers to the percentage of visitors who enter the site from that page and then leave ("bounce") without triggering any other requests to the analytics server during that session. 
 - ExitRates: `float`. Exit rate of a web page. The value of "Exit Rate" feature for a specific web page is calculated as for all pageviews to the page, the percentage that were the last in the session. 
 - PageValues: `object`. Page values of each web page. The "Page Value" feature represents the average value for a web page that a user visited before completing an e-commerce transaction. `Identify values and change dtype accordingly.` 
 - SpecialDay: Special days like valentine etc. `float`. Closeness to a special date. `dtype correct`. For example, for Valentina’s day, this value takes a nonzero value between February 2 and February 12, zero before and after this date unless it is close to another special day, and its maximum value of 1 on February 8. 
 - Month: Month of the year. `object`. String to identify the month of the year. `clean`. 
 - OperatingSystems: Operating system used. `object`. `Try to explain the values`. 
 - Browser: Browser used. `float` 
 - Region: Region of the user. `object`. 
 - TrafficType: Traffic Type. `float`.  
 - VisitorType: Types of Visitor `object` 
 - Weekend: Weekend or not `object`, `Identify values and change dtype accordingly`.   
 - Revenue: Revenue will be generated or not `bool`.
     

## Data Investigation and Cleaning

- **Administrative**
    - 999.0 probably move to nan, can't be that visited 999 times the page
- **Administrative_Duration**
    - change object type to float
    - move -99 and -1.0 values to nan
- **Informational_Duration**
    - change object type to float
    - change -1.0 to nan
    - change None to nan
- **ProductRelated_Duration**
    - change -1.0 values to Nan
- **Page Values**
    - change object type to float
    - #Nan change to Nan
- **Month**
    - change MAY to May and March to Mar
- **OperatingSystems**
    - change -999 and ? to Nan
- **Browser**
    - change -1 to Nan or it's a separate category of a browser?
- **Region**
    - change Nan to np.nan
- **VisitorType**
    - unify more to other or change both to Nan?
- **Weekend**
    - change object type to bool
    - change weekdays to 0 and 1

In [ ]:
# Show all values of the dataset

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

### Administrative

In [ ]:
data.Administrative.unique()

In [ ]:
data.Administrative.value_counts()

In [ ]:
data['AnyPage'] = data['Administrative'] + data['Informational'] + data['ProductRelated']

In [ ]:
data['AnyPage'].value_counts(normalize= True).hist(bins=50)

In [ ]:
# Checking 999 values

data999 = data.loc[data.Administrative == 999.0]
print(len(data999))
data999

- 999.0 probably move to nan, can't be that visited 999 times the page

In [ ]:
# Percenatge of revenue per 999 values:

data999.Revenue.value_counts(normalize = True)*100

**Changing to Nan:**

In [ ]:
data.Administrative = data.Administrative.replace(999.0, np.nan)

In [ ]:
data.Administrative.value_counts()

### Administrative Duration

In [ ]:
data.Administrative_Duration.unique()

In [ ]:
data.Administrative_Duration.value_counts()

In [ ]:
# Checking negative values (ignoring nans)

data_negative = data.loc[data.Administrative_Duration.str.contains('-', na = False)]
print(len(data_negative))
data_negative

- move negative values to nan, can't be duration negative

In [ ]:
# Percenatge of revenue per negative values:

data_negative.Revenue.value_counts(normalize = True)*100

**Changing to Nan:**

In [ ]:
data.Administrative_Duration = data.Administrative_Duration.replace(['-1.0', '-99'], np.nan)

In [ ]:
data.Administrative_Duration.value_counts()

### Informational

In [ ]:
data.Informational.unique()

In [ ]:
data.Informational.value_counts()

### Informational Duration

In [ ]:
data.Informational_Duration.unique()

In [ ]:
data.Informational_Duration.value_counts()

In [ ]:
# Checking neagitve values

data_negative = data.loc[data.Informational_Duration.str.contains('-', na = False)]
print(len(data_negative))
data_negative

- change negative to nan
- we have negative values in all 3 columns with duration

In [ ]:
# Percenatge of revenue per negative values:

data_negative.Revenue.value_counts(normalize = True)*100

In [ ]:
# checking none values:

data_none = data.loc[data.Informational_Duration == 'None']
print(len(data_none))
data_none

- change None values to Nan

In [ ]:
# Percenatge of revenue per none values:

data_none.Revenue.value_counts(normalize = True)*100

**Changing to Nan:**

In [ ]:
data.Informational_Duration = data.Informational_Duration.replace(['-1.0', 'None'], np.nan)

In [ ]:
data.Informational_Duration.value_counts()

### Product Related

In [ ]:
data.ProductRelated.unique()

In [ ]:
data.ProductRelated.value_counts()

### Product Related Duration

In [ ]:
data.ProductRelated_Duration.unique()

In [ ]:
data.ProductRelated_Duration.value_counts()

In [ ]:
# Checking negative values

data_negative = data.loc[data.ProductRelated_Duration < 0]
print(len(data_negative))
data_negative

- change negative values to Nan

In [ ]:
# Percenatge of revenue per negative values:

data_negative.Revenue.value_counts(normalize = True)*100

**Changing to Nan:**

In [ ]:
data.ProductRelated_Duration = data.ProductRelated_Duration.replace(-1.000000, np.nan)

In [ ]:
data.ProductRelated_Duration.value_counts()

### Bounce Rates

In [ ]:
data.BounceRates.unique()

In [ ]:
data.BounceRates.value_counts()

### Exit Rates

In [ ]:
data.ExitRates.unique()

In [ ]:
data.ExitRates.value_counts()

### Page Values

In [ ]:
data.PageValues.unique()

In [ ]:
data.PageValues.value_counts()

In [ ]:
# Checking #Nan values

data_nan = data.loc[data.PageValues == '#Nan']
print(len(data_nan))
data_nan

- #Nan change to Nan

In [ ]:
# Percenatge of revenue per #Nan values:

data_nan.Revenue.value_counts(normalize = True)*100

**Changing to Nan:**

In [ ]:
data.PageValues = data.PageValues.replace('#Nan', np.nan)

In [ ]:
data.ProductRelated_Duration.value_counts()

### Special Day

In [ ]:
data.SpecialDay.unique()

In [ ]:
data.SpecialDay.value_counts()

### Month

In [ ]:
data.Month.unique()

In [ ]:
data.Month.value_counts()

- change MAY to May and march to Mar

In [ ]:
data.Month = data.Month.str.replace('MAY', 'May').str.replace('March', 'Mar')

In [ ]:
data.Month.unique()

### Operating Systems

In [ ]:
data.OperatingSystems.unique()

In [ ]:
data.OperatingSystems.value_counts()

In [ ]:
# Checking -999 values

data999 = data.loc[data.OperatingSystems == '-999']
print(len(data999))
data999

- change -999 to Nan, also Browser is Nan when Operating systems is -999

In [ ]:
# Percenatge of revenue per -999 values:

data999.Revenue.value_counts(normalize = True)*100

In [ ]:
# Checking ? values

data_quest = data.loc[data.OperatingSystems == '?']
print(len(data_quest))
data_quest

- change ? to Nan

**Changing to Nan:**

In [ ]:
data.OperatingSystems = data.OperatingSystems.replace(['-999', '?'], np.nan)

In [ ]:
data.OperatingSystems.unique()

### Browser

In [ ]:
data.Browser.unique()

In [ ]:
data.Browser.value_counts()

In [ ]:
# Checking negative values

data_negative = data.loc[data.Browser < 0]
print(len(data_negative))
data_negative

- change -1 to Nan or it's a separate category of a browser?

In [ ]:
# Percenatge of revenue per negative values:

data_negative.Revenue.value_counts(normalize = True)*100

**Changing to Nan:**

In [ ]:
data.Browser = data.Browser.replace(-1.0, np.nan)

In [ ]:
data.Browser.unique()

### Region

In [ ]:
data.Region.unique()

In [ ]:
data.Region.value_counts()

In [ ]:
# Checking Nan values

data_nan = data.loc[data.Region == 'Nan']
print(len(data_nan))
data_nan

- change Nan to np.nan

In [ ]:
# Percenatge of revenue per Nan values:

data_nan.Revenue.value_counts(normalize = True)*100

**Changing to Nan:**

In [ ]:
data.Region = data.Region.replace('Nan', np.nan)

In [ ]:
data.Region.value_counts()

### Traffic type

In [ ]:
data.TrafficType.unique()

In [ ]:
data.TrafficType.value_counts()

### Visitor Type

In [ ]:
data.VisitorType.unique()

In [ ]:
data.VisitorType.value_counts()

In [ ]:
# Checking More and Other values

data_other = data.loc[(data.VisitorType == 'More') | (data.VisitorType == 'Other')]
print(len(data_other))
data_other

- unify more to other or change both to Nan?

In [ ]:
# Percenatge of revenue per more/other values:

data_other.Revenue.value_counts(normalize = True)*100

In [ ]:
data.VisitorType = data.VisitorType.str.replace('More', 'Other').fillna('Other')

**Changing to Nan:**

In [ ]:
data.VisitorType = data.VisitorType.replace('Other', np.nan)

In [ ]:
data.VisitorType.value_counts(normalize = True)*100

### Weekend

In [ ]:
data.Weekend.unique()

In [ ]:
data.Weekend.value_counts()

In [ ]:
# Checking values other than 0 and 1 values

data_week = data[data.Weekend.isin(['Monday', 'Sunday', 'Tuesday', 'Friday'])]
print(len(data_week))
data_week

- change weekdays to 0 and 1

In [ ]:
# Percenatge of revenue per weekdays values:

data_week.Revenue.value_counts(normalize = True)*100

In [ ]:
data.Weekend = data.Weekend.replace({'Tuesday' : '0.0', 'Monday': '0.0', 'Friday': '0.0', 'Sunday': '1.0'})

In [ ]:
data.Weekend.value_counts()

### Revenue

In [ ]:
data.Revenue.unique()

In [ ]:
data.Revenue.value_counts()

In [ ]:
# Checking the % of value counts

data.Revenue.value_counts(normalize = True)*100

### Data Types

In [ ]:
data.info()

In [ ]:
# Converting to appropriate data types

to_object = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 
             'Weekend', 'Revenue']

to_float = ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration', 'ProductRelated', 
            'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 
            'AnyPage']

data[to_object] = data[to_object].astype('object')

data[to_float] = data[to_float].apply(pd.to_numeric, errors='coerce')

In [ ]:
data.info()

In [ ]:
# clean data (no errors, correct data types, with NaNs)

clean = data.copy()

In [ ]:
clean.info()

## Data visualization

In [ ]:
cols = clean[to_object].columns

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(20,20))
fig.subplots_adjust(hspace=0.5)
for ax, col in zip(axes.flatten(), cols):
    sns.countplot(data=clean, x=col, hue='Revenue', ax=ax)
    
# poner titulos
# todo tiene la misma distribucion comparando a revenue (will learn how to equilibrar) en los modelos cargar con y sin
# hay que stratificar
# hay que buscar mas que 80
# mnimizar parametros en confusion matrix, minimzar los falsos positivos

# do feature engineering 
# hacer downsampling

# model first with parametros por defecto
# hypertuning, probar muchos modelos, diferentes parametros
# elegir medida de error, buscar mas comunes para esta problema, check other kernels, projects
# repartir tres modelos cada uno
# can do loop to try different modelos

In [ ]:
numeric_columns = list(clean.select_dtypes(include='float64').columns)

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(20,20))
fig.subplots_adjust(hspace=0.5)
for ax, col in zip(axes.flatten(), numeric_columns):
    sns.boxplot(data=clean, x=col, hue='Revenue', ax=ax)
    #sns.swarmplot(data=clean.Revenue, x=col, ax=ax) #put revenue con True
    
# How to have hue in outliers?

## Data Transformation

### Transforming columns

In [ ]:
# Adding a column from VisitorType transforming it's content into boolean: 1 is_returning, 0 is_new

clean['Visitor_isReturning'] = clean['VisitorType'].replace('Returning_Visitor', 1).replace('New_Visitor', 0)

- We can see in the visualization part that some of our principal dataset columns such as Operating Systems, Browser and TrafficType doesn't contain much information:


      - Operating Systems: 5-8 values don't give much information, there is no relation to Revenue True. 
      Change 5,6,7,8 to another category called 'other'.
  
  
      - Browser: Values 3, 7, 9, 11, 12, 13 change to 'other' category.
  
  
      - Traffic Type: 7, 12 and 14 to 19 change to 'other' category. 

**First of all, we are going to make a copy of the dataset via the .copy() method to ensure that the original DataFrame stays intact.**

In [ ]:
clean_datatransf = clean.copy()

In [ ]:
clean_datatransf.head()

#### Operating Systems

    In Python, it's a good practice to typecast categorical features to a category dtype because they make the operations on such columns much faster than the object dtype

In [ ]:
clean_datatransf['OperatingSystems'] = clean_datatransf['OperatingSystems'].astype('category')

In [ ]:
clean_datatransf.OperatingSystems.dtypes

    Let's apply a lambda function to classify our relevant values. After the visualization we can see that in Operating Systems we have categorical variables with few values and it's possible to reagrupate these values in a categorical variable called 'other'

In [ ]:
op_no_relevant_values = [5,6,7,8]

In [ ]:
clean_datatransf.OperatingSystems.apply(lambda x: 'other' if x in op_no_relevant_values else x)

        Applying a lambda function, we can see the 'other' values. If we do a .value_counts() can certify that our values are the same values than before. So, we are maintaining the data but with a better classification.

In [ ]:
clean_datatransf.OperatingSystems.value_counts()

        We can apply the same reasoning for the other categorical columns we want to transform

#### Browser

In [ ]:
clean_datatransf['Browser'] = clean_datatransf['Browser'].astype('category')

In [ ]:
clean_datatransf.Browser.dtypes

In [ ]:
brwsr_no_relevant_values = [3,7,9,11,12,13]

In [ ]:
clean_datatransf.Browser.apply(lambda x: 'other' if x in brwsr_no_relevant_values else x)

#### Traffic Type

In [ ]:
clean_datatransf['TrafficType'] = clean_datatransf['TrafficType'].astype('category')

In [ ]:
clean_datatransf.TrafficType.dtypes

In [ ]:
ttype_no_relevant_values = [7,12,14,15,16,17,18,19]

In [ ]:
clean_datatransf.TrafficType.apply(lambda x: 'other' if x in ttype_no_relevant_values else x)

### Data encoding

In [ ]:
# Filling missing values with ffill

clean_no_nan = clean.fillna(method='ffill').copy()

In [ ]:
# LABEL ENCODING: revenue and weekend to 0 and 1

# creating instance of labelencoder
labelencoder = LabelEncoder()

# Assigning numerical values and storing in another column

cols_label_enc = ['Revenue', 'Weekend']

for col in cols_label_enc:
    clean_no_nan[col+'_enc'] = labelencoder.fit_transform(clean_no_nan[col])

In [ ]:
clean_no_nan.head()

In [ ]:
# ONE HOT ENCODING for Month, OperatingSystems and Region

appended_dfs = []

columns = ['Month', 'OperatingSystems', 'Region']

for col in columns:
    # Creating instance of one-hot-encoder
    enc = OneHotEncoder(handle_unknown='ignore')
    # Passing columns
    enc_df = pd.DataFrame(enc.fit_transform(clean_no_nan[[col]]).toarray())
    # Getting values for the names of the columns
    enc_df.columns = enc.get_feature_names()+col
    # Appenign all new dfs to one list
    appended_dfs.append(enc_df)
# mergeing with main df
df_encoded = clean_no_nan.join(appended_dfs) 
df_encoded.head()

In [ ]:
df_encoded.info()

## Missing Values

In [ ]:
# already used it above

clean_no_nan = clean.fillna(method='ffill').copy()

## Correlation

In [ ]:
sns.set(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(clean_no_nan.corr(), dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(16, 12))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(clean_no_nan.corr(), mask=mask, cmap=cmap, vmax=.9, center=0,
            square=True, linewidths=.5, annot = True, cbar_kws={"shrink": .5})

plt.show()

In [ ]:
# print(f"After filling the NaN values, we can observe that our correlations vary in a {round(clean.corr()['Revenue'].sort_values(ascending=False) - clean_no_nan.corr()['Revenue'].sort_values(ascending=False), 3).sum() * 100}%")

In [ ]:
clean.corr()['Revenue'].sort_values(ascending=False)

### Correlation insights:

Make new column synthetizing the values of the non-correlated columns

# Individual work to organize

### Administrative_duration

- Data type: Categorical, ´object´. Necessary change to float.
- There are some values with multiple dots. Necessary to delete these dots.
- Missing Values: There are 14 missing values, which is 0.11% out of all data. Will fill this cells with median
- 47.5% of sessions are coming from the category 0.0 of the Administrative_Duration

In [ ]:
# Missing values

NA_values(data.Administrative_Duration)

In [ ]:
# Fixing problems
# Drop dots from cells


In [ ]:
# Distribution

print(data.Administrative_Duration.value_counts())

plt.style.use('seaborn')
fig, ax = plt.subplots(1, figsize=(8,6))
ax1 = data.Informational.hist()
plt.title("Administrative Duration - distribution")
plt.show()


In [ ]:
percent_info_0 = data.Administrative_Duration.value_counts()[0]/len(data)*100  
print(f'% of sessions in the category 0.0 of the Administrative_Duration pages: {round(percent_info_0,1)}')

### BounceRates

- Data type: Numerical, ‘float‘. Dtype change not necessary.
- Missing Values: There are 14 missing values, which is 0.11% out of all data. Will fill this cells with median
- 44.7% of sessions are coming from the category 0.0 of the BounceRates

In [ ]:
# Missing values

NA_values(data.BounceRates)

In [ ]:
# Distribution

print(data.BounceRates.value_counts())

plt.style.use('seaborn')
fig, ax = plt.subplots(1, figsize=(8,6))
ax1 = data.Informational.hist()
plt.title("BounceRates - distribution")
plt.show()

In [ ]:
percent_info_0 = data.BounceRates.value_counts()[0]/len(data)*100  
print(f'% of sessions in the category 0.0 of the BounceRates pages: {round(percent_info_0,1)}')

### OperatingSystems

In [ ]:
# checking the no. of OSes each user is having??

- Data type: Numerical, ‘float‘. Dtype change not necessary.
- Missing Values: There are 1 missing values, which is 0.01% out of all data. Will fill this cell with median
- 53.4% of sessions are coming from the category 2.0 of the OperatingSystems

In [ ]:
# Missing values

NA_values(data.OperatingSystems)

In [ ]:
# transform 999 and ? to NaN

data.OperatingSystems.fillna(method= 'ffill')

In [ ]:
print(data.OperatingSystems.value_counts())

plt.style.use('seaborn')
fig, ax = plt.subplots(1, figsize=(8,6))
ax1 = data.Informational.hist()
plt.title("OperatingSystems - distribution")
plt.show()

In [ ]:
percent_info_2 = data.OperatingSystems.value_counts()[2]/len(data)*100  
print(f'% of sessions in the category 2.0 of the OperatingSystems pages: {round(percent_info_2,1)}')

### Weekend

- Data type: Bool. Dtype change not necessary.
- Missing Values: There are no missing values.
- 76.7% of sessions are coming from the category False of the Weekend

In [ ]:
# Missing values

NA_values(data.Weekend)

In [ ]:
print(data.Weekend.value_counts())

plt.style.use('seaborn')
fig, ax = plt.subplots(1, figsize=(8,6))
ax = data.Informational.hist()
plt.title("Weekend - distribution")
plt.show()

In [ ]:
percent_info_false = data.Weekend.value_counts()[0]/len(data)*100  
print(f'% of sessions in the category False of the Weekend pages, not in weekend: {round(percent_info_0,1)}')

### Kristina:

 - **Informational**
  - Data type: Categorical, float. No changes
  - Missing Values: There are 14 missing values, which is 0.11% out of all data. It will be interpolated via a ffill, in order to preserve the actual distribution of the values.
  - 78.7% of sessions are coming from the category 0.0 of the Informational pages
  - No other changes are needed
 - **ExitRates**
  - Data type: Numerical, float. No changes
  - Missing Values: There are 14 missing values, which is 0.11% out of all data. It will be filled with the median
  - Outliers: There are 1094 outliers, which is 8.87% of all data. The majority of the outliers are falling under the FALSE revenue category and under the 0.2 value of the Exit rates. Additional column will be created to be able to filter out the outliers if needed: exitrates_outliers (boolean values).
 - **Browser**
  - Data type: Categorical, integer. No changes
  - Missing Values: There are 100 missing values, which is 0.8% out of all data. It will be interpolated via a ffill, in order to preserve the actual distribution of the values.
  - The most popular browser is 2. The usage share is very similar to the standard Usage share of all browsers. Later on, in the data visualization, we will show more insights on this
  - There is one category identified with anegative number (-1.0) not sure if we need to change it.
  - No other changes are needed
 - **Revenue**
  - Data type: categorical, boolean. No changes
  - There are 2 categories, True and False
  - 84.53% of data falls under FALSE category of Revenue. Since it's a target column, the data will need to be equilibrated.
  - Trying to plot as kind='bar' loads too long, that's why use hist()

#### Informational

In [ ]:
# Missing values

NA_values(data.Informational)

# Filling missing values

data.Informational.fillna(method='ffill', inplace=True)

# Distribution

print(data.Informational.value_counts())

plt.style.use('seaborn')
fig, ax = plt.subplots(1, figsize=(8,6))
ax = data.Informational.hist()
plt.title("Informational - distribution")
plt.show()

# % of sessions in category 0.0 

percent_info_0 = data.Informational.value_counts()[0]/len(data)*100  
print(f'% of sessions in the category 0.0 of the Informational pages: {round(percent_info_0,1)}')

#### Exit Rates

In [ ]:
# Missing values

NA_values(data.ExitRates)

# Filling missing values

data.ExitRates = data.ExitRates.fillna(data.ExitRates.median())

# Distribution

fig, ax = plt.subplots(1, figsize=(8,6))
plt.style.use('seaborn')
ax = data.ExitRates.hist()
plt.title("ExitRates - distribution")
plt.show()

# Outliers

fig, ax = plt.subplots(1, figsize=(8,6))
plt.style.use('seaborn')
ax = data.boxplot('ExitRates')
plt.title("ExitRates - outliers")
plt.show()

iqr(data, data.ExitRates)

# Checking the distribution of outliers regarding the target column - Revenue

exitrates_outliers = data.loc[(data.ExitRates > 0.104) | (data.ExitRates < -0.039)]
print(exitrates_outliers.Revenue.value_counts())

# Checking the top exit rate values of the outliers:

print(exitrates_outliers.ExitRates.value_counts())

# Creating a new column in the dataset to indicate exit rate outliers:
# Only using the upper IQR because the lower is a negative value and we don't have values lower tahn 0.

data['exitrates_outliers']  = data['ExitRates'].apply(lambda x: 'TRUE' if x > 0.104 else 'FALSE')

# Converting column to boolean

mapa = {'TRUE': True, 'FALSE': False}
data['exitrates_outliers'] = data['exitrates_outliers'].map(mapa)

#### Browser

In [ ]:
# Missing values

NA_values(data.Browser)

# Filling missing values

data.Browser.fillna(method='ffill', inplace=True)

# Distribution

print(data.Browser.value_counts(normalize = True)*100) 

fig, ax = plt.subplots(1, figsize=(8,6))
plt.style.use('seaborn')
ax = data.Browser.hist()
plt.title("Browser - distribution")
plt.show()

# Creating a dataset of the standard usage share of all browsers and of mobile browsers

usage_share_browsers = pd.DataFrame({'Chrome': 64.92, 'Safari': 15.97, 'Firefox': 4.33, 'Samsung_Internet': 3.29, 
                                     'UC': 2.94, 'Opera': 2.34, 'Edge': 2.05, 'IE': 1.98, 'AOSP': 0.59, 'Others': 1.59}, 
                                    index = [0]).T
colnames = ['standard_usage_all']
usage_share_browsers.columns = colnames
print(usage_share_browsers)

usage_share_browsers_mob = pd.DataFrame({'Chrome': 63.80, 'Safari': 19.70, 'Firefox': 0.35, 'Samsung_Internet': 6.27, 
                                     'UC': 5.33, 'Opera': 2.48, 'Others': 2.07}, 
                                    index = [0]).T
colnames = ['standard_usage_mobile']
usage_share_browsers_mob.columns = colnames
print(usage_share_browsers_mob)

#### Revenue

In [ ]:
# Missing values

NA_values(data.Revenue)

# Distribution

print(data.Revenue.value_counts())

# To plot a boolean variable, converting it to numeric

fig, ax = plt.subplots(1, figsize=(8,6))
plt.style.use('seaborn')
ax = data.Revenue.astype(float).hist()
plt.title("Revenue - distribution")
plt.show()

# How much data under False

print(f' {round(data.Revenue.value_counts(normalize = True)[0]*100, 2)}% of data falls under FALSE category of Revenue')

### Isaac:

    - Informational_Duration: There is a 'None' value so for this reason the type is an object.. We should change it to 0 and convert all the Series to float. In terms of missing values we got it covered (0.11%). We will fill them with the mean. 0.0% of outliers.
    - PageValues. Same as Informationa_Duration, we got an object called (#Nan). We should also change it and convert all the Series to float. Same for the missing values. 0.0% of outliers.
    - Region. There are only 9 regions. There are some with 'nan' or 'Nan'. My proposal is to remove whole cell as it will end up with bad results.

In [ ]:
# Informational_Duration column

data[data['Informational_Duration'] == 'None'] = 0
data["Informational_Duration"] = pd.to_numeric(data["Informational_Duration"])

NA_values(data["Informational_Duration"])
iqr(data, data["Informational_Duration"])

In [ ]:
# PageValues column

data[data['PageValues'] == '#Nan'] = 0
data["PageValues"] = pd.to_numeric(data["PageValues"])

NA_values(data["PageValues"])
iqr(data, data["PageValues"])

In [ ]:
# Region column

#data[data['Region'] == 'nan']
#data[data['Region'] == 'Nan']

#data["Region"] = pd.to_numeric(data["Region"])
data.Region.value_counts()



### Pau:

    - ProductRelated:

    - SpecialDay:

    - TrafficType:


In [ ]:
# ProductRelated column

# 1.09% NA --> filling method: median?:

NA_values(data.ProductRelated) 

data.ProductRelated.fillna(method='ffill', inplace=True)

# dtype transformation: float to int

In [ ]:
# Histogram | Outliers:
fig, ax = plt.subplots(1, figsize=(8,6))
plt.style.use('seaborn')
ax = data.ProductRelated.astype(float).hist()
plt.title("ProductRelated - distribution")
plt.show()

In [ ]:
data.ProductRelated.describe()

In [ ]:
# SpecialDay column

# 0.0% NA --> no filling needed:

NA_values(data.SpecialDay) 

# dtype correct (float)

In [ ]:
# Histogram | Outliers:
fig, ax = plt.subplots(1, figsize=(8,6))
plt.style.use('seaborn')
ax = data.SpecialDay.hist()
plt.title("SpecialDay - distribution")
plt.show()

In [ ]:
data.SpecialDay.value_counts()

In [ ]:
# TrafficType column

# 0.97% NA --> filling method: median?:

NA_values(data.TrafficType) 

data.TrafficType.fillna(method='ffill', inplace=True)

# dtype transformation: float to int

In [ ]:
# Histogram | Outliers:
fig, ax = plt.subplots(1, figsize=(8,6))
plt.style.use('seaborn')
ax = data.TrafficType.hist()
plt.title("TrafficType - distribution")
plt.show()

In [ ]:
data.TrafficType.value_counts()

### Sosa:

    - Administrative:
        Due to the low number of missing values (0.28%), we'll fill them with the median of the values since its 
        impact won't be noticeable. We can't use mean because each value represents a category and should be
        assigned to an existing value. 
        Also, we can also see that the distribution of the values is a logarithmic one, left-skewed.
        There's 100 values 999. These are obvious errors. They're distributed evenly through the column. To
        fix this, we'll transform this 999 values into NaNs to interpolate them via a ffill, in order to preserve
        the actual distribution of the values.
        
    - ProductRelated_Duration
        In this column, we have negative values. The time of a person staying in a webpage cannot be
        negative, so we assume there's a error on the lecture. We'll be replacing them with '0' value.
        We have just two pronounced outliers. 
        Additionaly, we'll assing our NAs, 0,14% of the values, to the mean values since our continuous numeric
        values are equally distributed.
        
        
    - Month
        We have no NA values, but some wrong strings to classify the months. We'll fix this with Regex.
        No more transformation needed.
        
    - VisitorType
        We have four types of visitors: Returning, New, Other and More. In this case, we'll reduce the group
        to three types: Returning, New and Other, by merging the More column to Other.
        For the NA values, we have no way to know if the visitor is Returning or New, so we'll also group them
        with the 'Other' values.

In [ ]:
## Administrative column

print(NA_values(data.Administrative))

data.Administrative = data.Administrative.fillna(data.Administrative.median())


data[data['Administrative'] == 999] = None

data.Administrative.fillna(method='ffill', inplace=True)

iqr(data, data.Administrative)



In [ ]:
## ProductRelated_Duration column

print(NA_values(data.ProductRelated_Duration))

# Fixing outliers:

outliers = data.ProductRelated_Duration.sort_values(ascending=False)[:2]

data.query('ProductRelated_Duration == @outliers')['ProductRelated_Duration'] = data.ProductRelated_Duration.mean()

# Fixing the NA values:

data.ProductRelated_Duration = data.ProductRelated_Duration.fillna(data.ProductRelated_Duration.mean())

## ??? IQR for logarithmic distributions, are there outliers?

In [ ]:
## Month column

NA_values(data.Month)

# Fixing strings:

data.Month = data.Month.str.replace('MAY', 'May').str.replace('March', 'Mar')

print(data.Month.value_counts())

print(data.Month.fillna(method='ffill').value_counts())

In [ ]:
## VisitorType column

NA_values(data.VisitorType)

data.VisitorType = data.VisitorType.str.replace('More', 'Other').fillna('Other')

## ver si la columna tiene importancia, probar otras formas de tratar 'Other'

### MODEL APPLICATION : K-Nearest Neighbors¶

Training and Testing Split: 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.loc[:,
                                   data.columns != 'Revenue'],
                                   data.Revenue,
                                   random_state=0)

In [ ]:
print('data shape:', data.shape)
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

In [ ]:
data.shape[0] == (X_train.shape[0] + X_test.shape[0])

Training Model:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neighbors = 1 

knn = KNeighborsClassifier(n_neighbors = neighbors)

In [ ]:
knn.fit(X_train, y_train)

Evaluating the Model with test set:

In [ ]:
y_pred = knn.predict(X_test)

print('Test set score: {:.2f}'.format(np.mean(y_pred == y_test)))

Prediction:

In [ ]:
# X_new
# prediction = knn.predict(X_new)